In [1]:
import os

list_file = sorted(list(os.walk('./file/6.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['type_1.Alignment Cam Bolt Assortment.xlsx',
 'type_10.Alignment Caster Wedge Multi-Pack.xlsx',
 'type_100.Toe Compensator Link.xlsx',
 'type_101.Torsion Bar Key - Adjustable.xlsx',
 'type_102.Torsion Bar Load Kit.xlsx',
 'type_103.Torsion Bar Mount.xlsx',
 'type_104.Track Bar.xlsx',
 'type_105.Track Bar Bushing.xlsx',
 'type_106.Trailing Arm.xlsx',
 'type_107.Trailing Arm Bushing.xlsx',
 'type_108.Transverse Spring Isolator Pad.xlsx',
 'type_109.Universal Joint.xlsx',
 'type_11.Alignment Caster%2FCamber Bushing.xlsx',
 'type_110.Universal Joint Strap Kit.xlsx',
 'type_111.Universal Joint U-Bolt Kit.xlsx',
 'type_112.Watts Link.xlsx',
 'type_113.Wheel Bearing.xlsx',
 'type_114.Wheel Bearing and Hub Assembly.xlsx',
 'type_115.Wheel Hub Repair Kit.xlsx',
 'type_12.Alignment Caster%2FCamber Bushing Kit.xlsx',
 'type_13.Alignment Caster%2FCamber Kit.xlsx',
 'type_14.Alignment Caster%2FCamber Shim.xlsx',
 'type_15.Alignment Shim.xlsx',
 'type_16.Alignment Shim Multi-Pack.xlsx',
 'type_17.A

In [2]:
file = list_file[29]

file

'type_21.Ball Joint.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/6.part/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Part Number'].tolist()

总数量：529


['K100054',
 'K100056',
 'K100057',
 'K100071',
 'K100107',
 'K100108',
 'K100115',
 'K100116',
 'K100140',
 'K100198',
 'K100199',
 'K100311',
 'K100312',
 'K100313',
 'K100314',
 'K100315',
 'K100316',
 'K100317',
 'K100328',
 'K100330',
 'K100331',
 'K100332',
 'K100398',
 'K100424',
 'K3082',
 'K3083',
 'K3134T',
 'K3134T005',
 'K3137T',
 'K3161T',
 'K3161T005',
 'K3185',
 'K3185T005',
 'K3199',
 'K500004',
 'K500005',
 'K500006',
 'K500007',
 'K500008',
 'K500012',
 'K500013',
 'K500014',
 'K500015',
 'K500016',
 'K500018',
 'K500030',
 'K500031',
 'K500032',
 'K500033',
 'K500034',
 'K500035',
 'K500036',
 'K500038',
 'K500041',
 'K500043',
 'K500044',
 'K500049',
 'K500050',
 'K500056',
 'K500057',
 'K500060',
 'K500062',
 'K500063',
 'K500064',
 'K500065',
 'K500066',
 'K500067',
 'K500069',
 'K500070',
 'K500072',
 'K500073',
 'K500074',
 'K500079',
 'K500082',
 'K500084',
 'K500087',
 'K500088',
 'K500090',
 'K500100',
 'K500101',
 'K500102',
 'K500103',
 'K500108',
 'K500109

In [5]:
a = 217

a

217

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

b = 0
while True:
    b += 1
    print(b)
    try:
        resp = requests.get(f'''https://www.drivparts.com/driv/partfinder/api.catalog.applications?brand=corporate&locale=en_US&country_code=US&part_number={input_.loc[a, 'Part Number']}&brand_code={input_.loc[a, 'Brand']}''',
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(10, 10))

        if resp.status_code == 200:
            json_ = resp.json()
            
            list_vehicle = json_['application_group_list']['PASSENGER CAR & LIGHT TRUCK']['applications']
            if list_vehicle:
                break
    except KeyboardInterrupt:
        break
    except:
        continue

resp

1
2
3


<Response [200]>

In [8]:
df_temp = pd.DataFrame()
for vehicle in list_vehicle:
    list_key = ['make',
                'model',
                'engine_base_value',
                'years',
                'position',
                'qty',
                'category',
                'drive',
                'engine_base',
                'engine_base_value',
                'engine_version',
                'engine_version_value',
                'engine_designation',
                'engine_vin']
    
    for key in vehicle:
        if key not in list_key:
            print('fuck')
        
    df_temp_temp = pd.DataFrame({'No': input_.loc[a, 'No'],
                                 'Url': input_.loc[a, 'Url'],
                                 'Make': vehicle['make']['value'].strip(),
                                 'Model': vehicle['model']['value'].strip(),
                                 'Engine': vehicle['engine_base']['engine_base_value'].strip() if 'engine_base' in vehicle else vehicle['engine_base_value'].strip(),
                                 'Engine VIN': vehicle['engine_vin']['value'].strip() if 'engine_vin' in vehicle else '',
                                 'Engine Desg.': vehicle['engine_designation']['value'].strip() if 'engine_designation' in vehicle else '',
                                 'Year': vehicle['years'],
                                 'Description': vehicle['category']['part_type_value'].strip(),
                                 'Position': '' if 'position' not in vehicle or vehicle['position']['value'].strip() == 'N/A' else vehicle['position']['value'].strip(),
                                 'Drive Wheel': vehicle['drive']['value'].strip() if 'drive' in vehicle else '',
                                 'Veh. Qty.': vehicle['qty']})
    
    df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Url,Make,Model,Engine,Engine VIN,Engine Desg.,Year,Description,Position,Drive Wheel,Veh. Qty.
0,447,https://www.drivparts.com/part-details.html?br...,Audi,A3,,,,2009,Ball Joint,Front Left Lower,FWD,1
1,447,https://www.drivparts.com/part-details.html?br...,Audi,A3,,,,2009,Ball Joint,Front Left Lower,FWD,1
2,447,https://www.drivparts.com/part-details.html?br...,Audi,A3,,,,2010,Ball Joint,Front Left Lower,FWD,1
3,447,https://www.drivparts.com/part-details.html?br...,Audi,A3,,,,2011,Ball Joint,Front Left Lower,FWD,1
4,447,https://www.drivparts.com/part-details.html?br...,Audi,A3,,,,2012,Ball Joint,Front Left Lower,FWD,1
...,...,...,...,...,...,...,...,...,...,...,...,...
212,447,https://www.drivparts.com/part-details.html?br...,Volkswagen,Tiguan,,,,2017,Ball Joint,Front Left Lower,,1
213,447,https://www.drivparts.com/part-details.html?br...,Volkswagen,Tiguan Limited,,,,2017,Ball Joint,Front Left Lower,,1
214,447,https://www.drivparts.com/part-details.html?br...,Volkswagen,Tiguan Limited,,,,2017,Ball Joint,Front Left Lower,,1
215,447,https://www.drivparts.com/part-details.html?br...,Volkswagen,Tiguan Limited,,,,2018,Ball Joint,Front Left Lower,,1


In [9]:
crawler_status = 'ok'

crawler_status

'ok'

In [10]:
if crawler_status == 'error':
    output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

output_error

""


In [11]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No', 'Make', 'Model', 'Engine', 'Year'],
                                                ascending=[True, True, True, True, False],
                                                ignore_index=True)
    output_correct.to_excel('./test_vehicle.xlsx', index=False)

output_correct

,No,Url,Make,Model,Engine,Engine VIN,Engine Desg.,Year,Description,Position,Drive Wheel,Veh. Qty.
0,447,https://www.drivparts.com/part-details.html?br...,Audi,A3,,,,2013,Ball Joint,Front Left Lower,FWD,1
1,447,https://www.drivparts.com/part-details.html?br...,Audi,A3,,,,2012,Ball Joint,Front Left Lower,FWD,1
2,447,https://www.drivparts.com/part-details.html?br...,Audi,A3,,,,2011,Ball Joint,Front Left Lower,FWD,1
3,447,https://www.drivparts.com/part-details.html?br...,Audi,A3,,,,2010,Ball Joint,Front Left Lower,FWD,1
4,447,https://www.drivparts.com/part-details.html?br...,Audi,A3,,,,2009,Ball Joint,Front Left Lower,FWD,1
...,...,...,...,...,...,...,...,...,...,...,...,...
212,447,https://www.drivparts.com/part-details.html?br...,Volkswagen,Tiguan,,,,2009,Ball Joint,Front Left Lower,,1
213,447,https://www.drivparts.com/part-details.html?br...,Volkswagen,Tiguan Limited,,,,2018,Ball Joint,Front Left Lower,,1
214,447,https://www.drivparts.com/part-details.html?br...,Volkswagen,Tiguan Limited,,,,2018,Ball Joint,Front Left Lower,,1
215,447,https://www.drivparts.com/part-details.html?br...,Volkswagen,Tiguan Limited,,,,2017,Ball Joint,Front Left Lower,,1


In [12]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_vehicle_error.xlsx', index=False)

output_error

""
